# Loading data into pandas dataframe





In [57]:
import pandas as pd
import numpy as np

df = pd.read_csv("online_shoppers_intention_numbers.csv")

print(df.columns)

#splitting the Class variable and the features
X = df.drop(columns=['Revenue'])
Y = df['Revenue']

#Making different datasets based on the top 10 features for testing purposes
datasets = {}

datasets[str(len(df.columns))] = X
datasets['10'] = X.drop(columns =['Month', 'TrafficType', 'Informational_Duration', 'OperatingSystems', 'Weekend','Region','Browser'])
datasets['9'] = datasets['10'].drop(columns =['SpecialDay'])
datasets['8'] = datasets['9'].drop(columns = ['Administrative_Duration'])
datasets['7'] = datasets['8'].drop(columns = ['Informational'])
datasets['6'] = datasets['7'].drop(columns = ['VisitorType'])
datasets['5'] = datasets['6'].drop(columns = ['Administrative'])
datasets['4'] = datasets['5'].drop(columns = ['BounceRates'])
datasets['3'] = datasets['4'].drop(columns = ['ProductRelated_Duration'])
datasets['2'] = datasets['3'].drop(columns = ['ProductRelated'])
datasets['1'] = datasets['2'].drop(columns = ['ExitRates'])

#for dataset in datasets:
#    print(datasets[dataset])

print(Y.values)




Index(['Administrative', 'Administrative_Duration', 'Informational',
       'Informational_Duration', 'ProductRelated', 'ProductRelated_Duration',
       'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay', 'Month',
       'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType',
       'Weekend', 'Revenue'],
      dtype='object')
[0 0 0 ... 0 0 0]


In [58]:
from sklearn.manifold import TSNE 
import matplotlib.pyplot as plt 
import seaborn as sns 

def tsne_plot(x, y):

    sns.set(style ="whitegrid") 
      
    tsne = TSNE(n_components = 2, random_state = 0) 
      
    # Reducing the dimensionality of the data 
    X_transformed = tsne.fit_transform(x) 
  
    plt.figure(figsize =(12, 8)) 
      
    # Building the scatter plot 
    plt.scatter(X_transformed[np.where(y == 0), 0],  
                X_transformed[np.where(y == 0), 1], 
                marker ='o', color ='y', linewidth ='1', 
                alpha = 0.8, label ='Non-Buyer') 
    plt.scatter(X_transformed[np.where(y == 1), 0], 
                X_transformed[np.where(y == 1), 1], 
                marker ='o', color ='k', linewidth ='1', 
                alpha = 0.8, label ='Buyer') 
  
    # Specifying the location of the legend 
    plt.legend(loc ='best') 
      
    # Plotting the reduced data 
    plt.show() 

#tsne_plot(X, Y)

In [59]:

from sklearn.preprocessing import MinMaxScaler





for dataset in datasets:
    scaler = MinMaxScaler()
    datasets[dataset] = scaler.fit_transform(datasets[dataset])
    
#scaled_datasets['10'] = X_scaled.drop(columns = ['Month', 'Traffictype', 'Informational_Duration', 'OperatingSystems', 'Weekend','Region','Browser'])
#scaled_datasets['9'] = X10_scaled.drop(columns =['SpecialDay'])
#scaled_datasets['8'] = X9_scaled.drop(columns = ['Administrative_Duration'])
#scaled_datasets['7'] = X8_scaled.drop(columns = ['Informational'])
#scaled_datasets['6'] = X7_scaled.drop(columns = ['VisitorType'])
#scaled_datasets['5'] = X6_scaled.drop(columns = ['Administrative'])
#scaled_datasets['4'] = X5_scaled.drop(columns = ['BounceRates'])
#scaled_datasets['3'] = X4_scaled.drop(columns = ['ProductRelated_Duration'])
#scaled_datasets['2'] = X3_scaled.drop(columns = ['ProductRelated'])
#scaled_datasets['1'] = X2_scaled.drop(columns = ['ExitRates'])




#tsne_plot(X_scaled, Y)

In [62]:
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split  
from sklearn.metrics import accuracy_score, confusion_matrix 

# Splitting dataset into train set and test set.
def makepredictions(datasets):
    
    for dataset in datasets:
    
        X_train, X_test, y_train, y_test = train_test_split(datasets[dataset], Y, test_size = 0.2, random_state = 32342) 


        svmclf = SVC() 
        svmclf.fit(X_train, y_train) 

        y_pred_svmclf = svmclf.predict(X_test) 
  
        # Performance
        print('amount of features: ' + dataset)
        print('Accuracy : '+str(accuracy_score(y_test, y_pred_svmclf))) 
        print('Confusion Matrix: \n' + str(confusion_matrix(y_test,y_pred_svmclf)))
        tn, fp, fn, tp = confusion_matrix(y_test, y_pred_svmclf).ravel()
        print('True negatives: ' + str(tn) + '\n' + 'False positives: ' + str(fp) +  '\n' + 'False negatives: ' + str(fn) + '\n'+ 'True positives: ' + str(tp) + '\n')

makepredictions(datasets)

amount of features: 18
Accuracy : 0.8763179237631792
Confusion Matrix: 
[[2033   32]
 [ 273  128]]
True negatives: 2033
False positives: 32
False negatives: 273
True positives: 128

amount of features: 10
Accuracy : 0.8868613138686131
Confusion Matrix: 
[[2022   43]
 [ 236  165]]
True negatives: 2022
False positives: 43
False negatives: 236
True positives: 165

amount of features: 9
Accuracy : 0.8868613138686131
Confusion Matrix: 
[[2021   44]
 [ 235  166]]
True negatives: 2021
False positives: 44
False negatives: 235
True positives: 166

amount of features: 8
Accuracy : 0.8872668288726683
Confusion Matrix: 
[[2021   44]
 [ 234  167]]
True negatives: 2021
False positives: 44
False negatives: 234
True positives: 167

amount of features: 7
Accuracy : 0.8856447688564477
Confusion Matrix: 
[[2020   45]
 [ 237  164]]
True negatives: 2020
False positives: 45
False negatives: 237
True positives: 164

amount of features: 6
Accuracy : 0.8864557988645579
Confusion Matrix: 
[[2018   47]
 [ 233  1

In [53]:
from keras.layers import Input, Dense 
from keras.models import Model, Sequential 
from keras import regularizers 

scaler = MinMaxScaler()

X_scaled = scaler.fit_transform(X)
X_nonbuyer_scaled = X_scaled[Y == 0] 
X_buyer_scaled = X_scaled[Y == 1]


input_layer = Input(shape =(X.shape[1], )) 

#Encoder network 
encoded = Dense(100, activation ='tanh', 
activity_regularizer = regularizers.l1(10e-5))(input_layer) 
encoded = Dense(50, activation ='tanh', 
activity_regularizer = regularizers.l1(10e-5))(encoded) 
encoded = Dense(25, activation ='tanh', 
activity_regularizer = regularizers.l1(10e-5))(encoded) 
encoded = Dense(12, activation ='tanh', 
activity_regularizer = regularizers.l1(10e-5))(encoded) 
encoded = Dense(6, activation ='relu')(encoded) 

#Decoder network 
decoded = Dense(12, activation ='tanh')(encoded) 
decoded = Dense(25, activation ='tanh')(decoded) 
decoded = Dense(50, activation ='tanh')(decoded) 
decoded = Dense(100, activation ='tanh')(decoded) 

# Building the Output Layer 
output_layer = Dense(X.shape[1], activation ='relu')(decoded) 


# Defining the parameters of the Auto-encoder network 
autoencoder = Model(input_layer, output_layer) 
autoencoder.compile(optimizer ="adadelta", loss ="mse") 
  
# Training the Auto-encoder network 
autoencoder.fit(X_nonbuyer_scaled, X_nonbuyer_scaled,  
                batch_size = 1500, epochs = 10,  
                shuffle = True, validation_split = 0.2) 

hidden_representation = Sequential() 
hidden_representation.add(autoencoder.layers[0]) 
hidden_representation.add(autoencoder.layers[1]) 
hidden_representation.add(autoencoder.layers[2]) 
hidden_representation.add(autoencoder.layers[3]) 
hidden_representation.add(autoencoder.layers[4]) 

# Separating the points encoded by the Auto-encoder as normal and fraud 
nonbuyer_hidden_rep = hidden_representation.predict(X_nonbuyer_scaled) 
buyer_hidden_rep = hidden_representation.predict(X_buyer_scaled) 
  
# Combining the encoded points into a single table  
encoded_X = np.append(nonbuyer_hidden_rep, buyer_hidden_rep, axis = 0) 
y_nonbuyer = np.zeros(nonbuyer_hidden_rep.shape[0]) 
y_buyer = np.ones(buyer_hidden_rep.shape[0]) 
encoded_y = np.append(y_nonbuyer, y_buyer) 
  
# Plotting the encoded points 
#tsne_plot(encoded_X, encoded_y) 


Train on 8337 samples, validate on 2085 samples
Epoch 1/10
8337/8337 [==============================] - 0s 49us/step - loss: 3.2603 - val_loss: 2.8986
Epoch 2/10
8337/8337 [==============================] - 0s 4us/step - loss: 2.4118 - val_loss: 2.3104
Epoch 3/10
8337/8337 [==============================] - 0s 4us/step - loss: 2.0276 - val_loss: 2.0003
Epoch 4/10
8337/8337 [==============================] - 0s 4us/step - loss: 1.7992 - val_loss: 1.7913
Epoch 5/10
8337/8337 [==============================] - 0s 4us/step - loss: 1.6223 - val_loss: 1.6295
Epoch 6/10
8337/8337 [==============================] - 0s 4us/step - loss: 1.4849 - val_loss: 1.5040
Epoch 7/10
8337/8337 [==============================] - 0s 4us/step - loss: 1.4096 - val_loss: 1.3992
Epoch 8/10
8337/8337 [==============================] - 0s 4us/step - loss: 1.3182 - val_loss: 1.3143
Epoch 9/10
8337/8337 [==============================] - 0s 4us/step - loss: 1.2332 - val_loss: 1.2421
Epoch 10/10
8337/8337 [==========

In [55]:
from sklearn.linear_model import LogisticRegression 

print(len(y_buyer))
print(len(y_nonbuyer))
print(len(encoded_y))
# Splitting the encoded data for linear classification 
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(encoded_X, encoded_y, test_size = 0.2, random_state = 32342)
print(len(encoded_y))
# Building the logistic regression model 
lrclf = SVC() 
lrclf.fit(X_train_encoded, y_train_encoded) 
  
# Storing the predictions of the linear model 
y_pred_lrclf = lrclf.predict(X_test_encoded) 

# Performance
print('amount of features: ')
print('Accuracy : '+str(accuracy_score(y_test_encoded, y_pred_lrclf))) 
print('Confusion Matrix: \n' + str(confusion_matrix(y_test_encoded,y_pred_lrclf)))
tn, fp, fn, tp = confusion_matrix(y_test_encoded, y_pred_lrclf).ravel()
print('True negatives: ' + str(tn) + '\n' + 'False positives: ' + str(fp) +  '\n' + 'False negatives: ' + str(fn) + '\n'+ 'True positives: ' + str(tp) + '\n')

1908
10422
12330
12330
amount of features: 
Accuracy : 0.8913219789132197
Confusion Matrix: 
[[2038   46]
 [ 222  160]]
True negatives: 2038
False positives: 46
False negatives: 222
True positives: 160



In [56]:
#Denoising Autoencoder

#loading only images and not their labels
X_scaled = scaler.fit_transform(X)
X_nonbuyer_scaled = X_scaled[Y == 0] 
X_buyer_scaled = X_scaled[Y == 1]


X_nonbuyer_scaled = X_nonbuyer_scaled.astype('float32')/255
X_buyer_scaled = X_buyer_scaled.astype('float32')/255

X_nonbuyer_scaled = X_nonbuyer_scaled.reshape(len(X_nonbuyer_scaled), np.prod(X_nonbuyer_scaled.shape[1:]))
X_buyer_scaled = X_buyer_scaled.reshape(len(X_buyer_scaled), np.prod(X_buyer_scaled.shape[1:]))
X_nonbuyer_noisy = X_nonbuyer_scaled + np.random.normal(loc=0.0, scale=0.5, size=X_nonbuyer_scaled.shape)
X_nonbuyer_noisy = np.clip(X_nonbuyer_noisy, 0., 1.)
X_buyer_noisy = X_buyer_scaled + np.random.normal(loc=0.0, scale=0.5, size=X_buyer_scaled.shape)
X_buyer_noisy = np.clip(X_buyer_noisy, 0., 1.)
print(X_nonbuyer_noisy.shape)
print(X_buyer_noisy.shape)


autoencoder.fit(X_nonbuyer_noisy, X_nonbuyer_noisy,  
                batch_size = 16, epochs = 10,  
                shuffle = True, validation_split = 0.2) 

hidden_representation = Sequential() 
hidden_representation.add(autoencoder.layers[0]) 
hidden_representation.add(autoencoder.layers[1]) 
hidden_representation.add(autoencoder.layers[2]) 
hidden_representation.add(autoencoder.layers[3]) 
hidden_representation.add(autoencoder.layers[4]) 

# Separating the points encoded by the Auto-encoder as normal and fraud 
nonbuyer_hidden_rep = hidden_representation.predict(X_nonbuyer_noisy) 
buyer_hidden_rep = hidden_representation.predict(X_buyer_noisy) 
  
# Combining the encoded points into a single table  
encoded_X = np.append(nonbuyer_hidden_rep, buyer_hidden_rep, axis = 0) 
y_nonbuyer = np.zeros(nonbuyer_hidden_rep.shape[0]) 
y_buyer = np.ones(buyer_hidden_rep.shape[0]) 
encoded_y = np.append(y_nonbuyer, y_buyer) 

X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(encoded_X, encoded_y, test_size = 0.2, random_state = 32342)

lrclf.fit(X_train_encoded, y_train_encoded) 
y_pred_lrclf = lrclf.predict(X_test_encoded) 


# Performance
print('amount of features: ')
print('Accuracy : '+str(accuracy_score(y_test_encoded, y_pred_lrclf))) 
print('Confusion Matrix: \n' + str(confusion_matrix(y_test_encoded,y_pred_lrclf)))
tn, fp, fn, tp = confusion_matrix(y_test_encoded, y_pred_lrclf).ravel()
print('True negatives: ' + str(tn) + '\n' + 'False positives: ' + str(fp) +  '\n' + 'False negatives: ' + str(fn) + '\n'+ 'True positives: ' + str(tp) + '\n')

(10422, 17)
(1908, 17)
Train on 8337 samples, validate on 2085 samples
Epoch 1/10
8337/8337 [==============================] - 1s 71us/step - loss: 0.0767 - val_loss: 0.0664
Epoch 2/10
8337/8337 [==============================] - 1s 68us/step - loss: 0.0589 - val_loss: 0.0594
Epoch 3/10
8337/8337 [==============================] - 1s 68us/step - loss: 0.0527 - val_loss: 0.0531
Epoch 4/10
8337/8337 [==============================] - 1s 69us/step - loss: 0.0487 - val_loss: 0.0497
Epoch 5/10
8337/8337 [==============================] - 1s 69us/step - loss: 0.0465 - val_loss: 0.0528
Epoch 6/10
8337/8337 [==============================] - 1s 67us/step - loss: 0.0449 - val_loss: 0.0608
Epoch 7/10
8337/8337 [==============================] - 1s 67us/step - loss: 0.0441 - val_loss: 0.0466
Epoch 8/10
8337/8337 [==============================] - 1s 66us/step - loss: 0.0435 - val_loss: 0.0549
Epoch 9/10
8337/8337 [==============================] - 1s 67us/step - loss: 0.0432 - val_loss: 0.0596
Ep